In [ ]:
import pandas as pd
import re
import nltk
# nltk.download('vader_lexicon')
import warnings
# from pandas.core.common import SettingWithCopyWarning
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# Read the data
news = pd.read_csv('../textData/bac_news.csv')
news.dropna(inplace = True)
news.reset_index(drop=True, inplace=True)
news.rename(columns={'release_date':'date'}, inplace=True)
news.drop(inplace=True, columns=['category', 'provider', 'url', 'article_id'], axis=1)
news.head()

,Unnamed: 0,id,ticker,title,content,date
0,41710,263225,BAC,U S Investors Go All In On Equity ETFs,At the end of last week analysts at Bank of A...,2016-12-20
1,41711,263226,BAC,It s The Big Banks Time To Shine,A couple of days after Donald Trump was electe...,2016-12-23
2,41712,263227,BAC,7 Reasons To Invest In Bank Of America Heading...,There are a few times in the life of an invest...,2016-12-29
3,41713,263228,BAC,Outlook 2017 Part II Stocks Bonds Inflatio...,Without question 2016 was the year of the big...,2016-12-30
4,41714,263229,BAC,Financial Crash Fallout Continues U S Sues B...,If you saw your Barclays LON BARC stock take...,2017-01-01


In [10]:
news_clean = news[['content', 'date']]
texts = news_clean['content']

# Generate sentiment score
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
texts

0       At the end of last week  analysts at Bank of A...
1       A couple of days after Donald Trump was electe...
2       There are a few times in the life of an invest...
3       Without question  2016 was the year of the big...
4       If you saw your Barclays  LON BARC  stock take...
                              ...                        
3146    The week beginning Feb  5 follows one during w...
3147    Investing com   U S  stocks soared on Monday  ...
3148    Investing com   U S  stocks rose on Thursday a...
3149    Investing com   U S  stock markets rallied in ...
3150    Investing com   European stocks were mixed on ...
Name: content, Length: 3151, dtype: object

In [7]:
SentimentAnalyzer(texts[0])

array([[0.21541387, 0.08111808, 0.7034681 ]], dtype=float32)

In [11]:
def SentimentAnalyzer(doc):
    pt_batch = tokenizer(doc,padding=True,truncation=True,max_length=512,return_tensors="pt")
    outputs = model(**pt_batch)
    pt_predictions = F.softmax(outputs.logits, dim=-1)
    return pt_predictions.detach().cpu().numpy()

sentiments = texts.apply(SentimentAnalyzer)
sentiments

0          [[0.21541387, 0.08111808, 0.7034681]]
1            [[0.211917, 0.06347466, 0.7246083]]
2         [[0.13194922, 0.09089943, 0.77715135]]
3        [[0.039089628, 0.85879165, 0.10211882]]
4         [[0.02020528, 0.83704627, 0.14274846]]
                          ...                   
3146        [[0.03661576, 0.16315323, 0.800231]]
3147       [[0.9043241, 0.02599773, 0.06967818]]
3148      [[0.8970054, 0.06550396, 0.037490707]]
3149      [[0.93853354, 0.02400052, 0.03746589]]
3150    [[0.026440836, 0.95894796, 0.014611267]]
Name: content, Length: 3151, dtype: object

In [14]:
news_clean['Negative'] = [0] * len(news_clean)
news_clean['Neutral'] = [0] * len(news_clean)
news_clean['Positive'] = [0] * len(news_clean)

for i in range(len(news_clean)):
    news_clean['Positive'][i] = sentiments[i][0][0]
    news_clean['Negative'][i] = sentiments[i][0][1]
    news_clean['Neutral'][i] = sentiments[i][0][2]

news_clean.sort_values(by = 'date', inplace = True)

# Pure sentiment score!!!
news_clean.tail()

C:\Users\mike9\AppData\Local\Temp\ipykernel_13636\168520558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_clean['Negative'] = [0] * len(news_clean)
C:\Users\mike9\AppData\Local\Temp\ipykernel_13636\168520558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_clean['Neutral'] = [0] * len(news_clean)
C:\Users\mike9\AppData\Local\Temp\ipykernel_13636\168520558.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

,content,date,Negative,Neutral,Positive
1049,By Silvia Aloisi Paola Arosio and Pamela Barb...,2016-12-26,0.485563,0.436859,0.077579
2,There are a few times in the life of an invest...,2016-12-29,0.090899,0.777151,0.131949
1050,By Sujata Rao and Andrey Ostroukh LONDON MOSCO...,2016-12-30,0.233180,0.135622,0.631199
3,Without question 2016 was the year of the big...,2016-12-30,0.858792,0.102119,0.039090
4,If you saw your Barclays LON BARC stock take...,2017-01-01,0.837046,0.142748,0.020205


In [15]:
news_clean_3 = news_clean[['date', 'Negative', 'Neutral', 'Positive']]
news_clean_mean = news_clean_3.groupby('date').mean()
news_clean_sum = news_clean_3.groupby('date').sum()

In [16]:
news_clean_mean

,Negative,Neutral,Positive
date,,,
2010-01-31,0.163153,0.800231,0.036616
2010-05-10,0.025998,0.069678,0.904324
2010-05-27,0.065504,0.037491,0.897005
2010-07-06,0.024001,0.037466,0.938534
2010-07-14,0.958948,0.014611,0.026441
...,...,...,...
2016-12-23,0.438628,0.464933,0.096439
2016-12-26,0.485563,0.436859,0.077579
2016-12-29,0.090899,0.777151,0.131949


In [17]:
news_clean_sum

,Negative,Neutral,Positive
date,,,
2010-01-31,0.163153,0.800231,0.036616
2010-05-10,0.025998,0.069678,0.904324
2010-05-27,0.065504,0.037491,0.897005
2010-07-06,0.024001,0.037466,0.938534
2010-07-14,0.958948,0.014611,0.026441
...,...,...,...
2016-12-23,1.754512,1.859734,0.385754
2016-12-26,0.485563,0.436859,0.077579
2016-12-29,0.090899,0.777151,0.131949


In [18]:
news_clean_mean.to_csv('news_FinBert_avg.csv')
news_clean_sum.to_csv('news_FinBert_sum.csv')